In [1]:
import pandas as pd
import numpy as np
import re
import os

# 1 匯入資料

## 1-1 匯入危機公司新聞

### 1-1-1 匯入TEJ危機公司新聞

In [2]:
tej_distress = pd.read_csv('data/2018_2017_tej_distress_0423.csv')
tej_distress.head()

,公司碼,年月,則次,序號,新聞內容
0,2474 可成,2017/12/29,01,10,可成旗下子公司可成科技( 宿遷) 及可功科技( 宿遷)28 日分別斥資6.2 億元及6.3 ...
1,1611 中電,2017/12/28,01,10,公開資訊觀測站重大訊息公告 ...
2,1611 中電,2017/12/28,01,11,(99)北市估字第000156號 ...
3,2062 橋椿,2017/12/27,01,10,針對珠海市人民政府發出暫時對重點工廠企業的限產限排令，要求12/26-12/29 配合限產限...
4,3027 盛達,2017/12/25,01,10,電源暨能源管理廠盛達電相中太陽光電系統市場商機可期，今年全力朝太陽光電系統專業設備商及服務商...


In [3]:
len(tej_distress)

3444

### 1-1-2 匯入UDN危機公司新聞

In [4]:
# 設定路徑，將該資料夾內所有檔案名稱列出來
Folder_Path = 'data/udn_distress/'
file_list = os.listdir(Folder_Path)
# file_list

In [5]:
udn_distress = pd.DataFrame(columns=['company', 'title', 'time', 'content'])

for file in file_list:
    df_temp = pd.read_csv('data/udn_distress/' + file)
    udn_distress = pd.concat([udn_distress, df_temp])
udn_distress = udn_distress.drop('title', axis=1).dropna()
udn_distress.head()

,company,time,content
0,2331 精英,【2009-04-13/經濟日報/C3版/市場焦點】,（\r\n）積極搶攻電信業自有品牌小筆電市場宣告開花結果。\r\n主管表示，\r\n近期拿下...
1,2331 精英,【2009-05-08/經濟日報/C3版/市場焦點】,（\r\n）營運出現大轉機，新任總經理許志宏昨（7）日上任，\r\n將再搶下大陸第二階段家電...
2,2331 精英,【2009-03-15/經濟日報/S07版/漲幅較大股】,（\r\n）受惠大陸山寨熱潮，今年筆記型電腦（NB）出貨量可望大增，配合台股破底翻，上周股價...
3,2331 精英,【2009-01-07/經濟日報/C6版/上市櫃公司2】,英特爾第二代學生電腦（Classmate PC）將在美國消費性電子展（CES）中亮相，採用A...
4,2331 精英,【2009-05-25/經濟日報/C3版/市場焦點】,包括台灣的威邁思、威達超舜、全球一動等全球WiMAX業者將陸續在下半年商轉服務，將帶動WiM...


In [6]:
len(udn_distress)

2589

### 1-1-3 （針對UDN）擷取日期，把日期格式轉換為yyyy/mm/dd

In [7]:
from datetime import date
import datetime

#測試一下

s = udn_distress['time'].iloc[4]
print(s)
match = re.search('\d+-\d+-\d+', s)
date = str(match.group(0))
date = date.replace('-', '/')
print(date)
date = datetime.datetime.strptime(date, '%Y/%m/%d').date()
# date = datetime.datetime.strptime(match.group(0), '%Y-%m-%d').date()
print(date)

【2009-05-25/經濟日報/C3版/市場焦點】
2009/05/25
2009-05-25


In [8]:
dates = []
for time in udn_distress['time']:
    s = time
    match = re.search('\d+-\d+-\d+', s)
    date = str(match.group(0))
    date = date.replace('-', '/')
    dates.append(date)
udn_distress['date'] = dates
# udn_distress.drop('time', axis=1, inplace=True)
udn_distress

,company,time,content,date
0,2331 精英,【2009-04-13/經濟日報/C3版/市場焦點】,（\r\n）積極搶攻電信業自有品牌小筆電市場宣告開花結果。\r\n主管表示，\r\n近期拿下...,2009/04/13
1,2331 精英,【2009-05-08/經濟日報/C3版/市場焦點】,（\r\n）營運出現大轉機，新任總經理許志宏昨（7）日上任，\r\n將再搶下大陸第二階段家電...,2009/05/08
2,2331 精英,【2009-03-15/經濟日報/S07版/漲幅較大股】,（\r\n）受惠大陸山寨熱潮，今年筆記型電腦（NB）出貨量可望大增，配合台股破底翻，上周股價...,2009/03/15
3,2331 精英,【2009-01-07/經濟日報/C6版/上市櫃公司2】,英特爾第二代學生電腦（Classmate PC）將在美國消費性電子展（CES）中亮相，採用A...,2009/01/07
4,2331 精英,【2009-05-25/經濟日報/C3版/市場焦點】,包括台灣的威邁思、威達超舜、全球一動等全球WiMAX業者將陸續在下半年商轉服務，將帶動WiM...,2009/05/25
...,...,...,...,...
227,1301 台塑,【2017-10-09/經濟日報/C2版/市場脈動】,元大投顧非科技產業分析師李侃奇看好\n集團中的\n（\n）、南亞，均給予「買進」評等；台化與...,2017/10/09
229,1301 台塑,【2017-09-12/經濟日報/C8版/權證】,（\n）是台灣最大的石化集團，主要經營產品包括塑膠原料、塑膠加工、精密化學和纖維製品等。隨著...,2017/09/12
231,1301 台塑,【2017-09-12/經濟日報/AA2版/上市櫃公司8月營收】,大小塑化公司8月營收均繳出漂亮成績單，其中，\n四寶中的\n（\n）8月營收，年增32.9%...,2017/09/12
233,1301 台塑,【2017-10-05/經濟日報/C6版/上市櫃公司】,中油年產35萬公噸乙烯的四輕完成換熱器設備搶修後，目前正加緊拉高開工率，即將步入產能全開盛況...,2017/10/05


In [9]:
# 把一些字串刪掉改空格
udn_distress['content'] = udn_distress['content'].astype(str).str.replace('\r' , ' ').str.replace('\t' , ' ').str.replace("\n", " ").str.replace(",", " ").str.replace('"' , ' ').str.replace("'" , ' ')
udn_distress['company'] = udn_distress['company'].astype(str).str.replace('\r' , ' ').str.replace('\t' , ' ').str.replace("\n", " ").str.replace(",", " ").str.replace('"' , ' ').str.replace("'" , ' ')
udn_distress

,company,time,content,date
0,2331 精英,【2009-04-13/經濟日報/C3版/市場焦點】,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...,2009/04/13
1,2331 精英,【2009-05-08/經濟日報/C3版/市場焦點】,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...,2009/05/08
2,2331 精英,【2009-03-15/經濟日報/S07版/漲幅較大股】,（ ）受惠大陸山寨熱潮，今年筆記型電腦（NB）出貨量可望大增，配合台股破底翻，上周股價上漲...,2009/03/15
3,2331 精英,【2009-01-07/經濟日報/C6版/上市櫃公司2】,英特爾第二代學生電腦（Classmate PC）將在美國消費性電子展（CES）中亮相，採用A...,2009/01/07
4,2331 精英,【2009-05-25/經濟日報/C3版/市場焦點】,包括台灣的威邁思、威達超舜、全球一動等全球WiMAX業者將陸續在下半年商轉服務，將帶動WiM...,2009/05/25
...,...,...,...,...
227,1301 台塑,【2017-10-09/經濟日報/C2版/市場脈動】,元大投顧非科技產業分析師李侃奇看好 集團中的 （ ）、南亞，均給予「買進」評等；台化與 化雖...,2017/10/09
229,1301 台塑,【2017-09-12/經濟日報/C8版/權證】,（ ）是台灣最大的石化集團，主要經營產品包括塑膠原料、塑膠加工、精密化學和纖維製品等。隨著油...,2017/09/12
231,1301 台塑,【2017-09-12/經濟日報/AA2版/上市櫃公司8月營收】,大小塑化公司8月營收均繳出漂亮成績單，其中， 四寶中的 （ ）8月營收，年增32.9%居冠，...,2017/09/12
233,1301 台塑,【2017-10-05/經濟日報/C6版/上市櫃公司】,中油年產35萬公噸乙烯的四輕完成換熱器設備搶修後，目前正加緊拉高開工率，即將步入產能全開盛況...,2017/10/05


In [10]:
udn_distress = udn_distress.loc[:, ['company', 'content', 'date']]
udn_distress

,company,content,date
0,2331 精英,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...,2009/04/13
1,2331 精英,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...,2009/05/08
2,2331 精英,（ ）受惠大陸山寨熱潮，今年筆記型電腦（NB）出貨量可望大增，配合台股破底翻，上周股價上漲...,2009/03/15
3,2331 精英,英特爾第二代學生電腦（Classmate PC）將在美國消費性電子展（CES）中亮相，採用A...,2009/01/07
4,2331 精英,包括台灣的威邁思、威達超舜、全球一動等全球WiMAX業者將陸續在下半年商轉服務，將帶動WiM...,2009/05/25
...,...,...,...
227,1301 台塑,元大投顧非科技產業分析師李侃奇看好 集團中的 （ ）、南亞，均給予「買進」評等；台化與 化雖...,2017/10/09
229,1301 台塑,（ ）是台灣最大的石化集團，主要經營產品包括塑膠原料、塑膠加工、精密化學和纖維製品等。隨著油...,2017/09/12
231,1301 台塑,大小塑化公司8月營收均繳出漂亮成績單，其中， 四寶中的 （ ）8月營收，年增32.9%居冠，...,2017/09/12
233,1301 台塑,中油年產35萬公噸乙烯的四輕完成換熱器設備搶修後，目前正加緊拉高開工率，即將步入產能全開盛況...,2017/10/05


In [11]:
udn_distress.to_csv("data/udn_distress_news.csv", index=False)

In [12]:
udn_distress = pd.read_csv("data/udn_distress_news.csv", encoding="utf-8")
udn_distress

,company,content,date
0,2331 精英,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...,2009/04/13
1,2331 精英,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...,2009/05/08
2,2331 精英,（ ）受惠大陸山寨熱潮，今年筆記型電腦（NB）出貨量可望大增，配合台股破底翻，上周股價上漲...,2009/03/15
3,2331 精英,英特爾第二代學生電腦（Classmate PC）將在美國消費性電子展（CES）中亮相，採用A...,2009/01/07
4,2331 精英,包括台灣的威邁思、威達超舜、全球一動等全球WiMAX業者將陸續在下半年商轉服務，將帶動WiM...,2009/05/25
...,...,...,...
2584,1301 台塑,元大投顧非科技產業分析師李侃奇看好 集團中的 （ ）、南亞，均給予「買進」評等；台化與 化雖...,2017/10/09
2585,1301 台塑,（ ）是台灣最大的石化集團，主要經營產品包括塑膠原料、塑膠加工、精密化學和纖維製品等。隨著油...,2017/09/12
2586,1301 台塑,大小塑化公司8月營收均繳出漂亮成績單，其中， 四寶中的 （ ）8月營收，年增32.9%居冠，...,2017/09/12
2587,1301 台塑,中油年產35萬公噸乙烯的四輕完成換熱器設備搶修後，目前正加緊拉高開工率，即將步入產能全開盛況...,2017/10/05


### 1-1-4 合併危機公司資料集為 distress_news

In [13]:
distress_news = pd.DataFrame(columns=['company', 'date', 'content'])
company = udn_distress['company']
company = company.append(tej_distress['公司碼'])
date = udn_distress['date']
date = date.append(tej_distress['年月'])
content = udn_distress['content']
content = content.append(tej_distress['新聞內容'])
distress_news['company'] = company
distress_news['date'] = date
distress_news['content'] = content

In [14]:
distress_news.head()

,company,date,content
0,2331 精英,2009/04/13,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...
1,2331 精英,2009/05/08,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...
2,2331 精英,2009/03/15,（ ）受惠大陸山寨熱潮，今年筆記型電腦（NB）出貨量可望大增，配合台股破底翻，上周股價上漲...
3,2331 精英,2009/01/07,英特爾第二代學生電腦（Classmate PC）將在美國消費性電子展（CES）中亮相，採用A...
4,2331 精英,2009/05/25,包括台灣的威邁思、威達超舜、全球一動等全球WiMAX業者將陸續在下半年商轉服務，將帶動WiM...


In [15]:
len(distress_news)

6033

In [16]:
distress_news.to_csv("data/tej_udn_distress_news.csv", index=False)
distress_news = pd.read_csv("data/tej_udn_distress_news.csv", encoding="utf-8")
distress_news

,company,date,content
0,2331 精英,2009/04/13,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...
1,2331 精英,2009/05/08,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...
2,2331 精英,2009/03/15,（ ）受惠大陸山寨熱潮，今年筆記型電腦（NB）出貨量可望大增，配合台股破底翻，上周股價上漲...
3,2331 精英,2009/01/07,英特爾第二代學生電腦（Classmate PC）將在美國消費性電子展（CES）中亮相，採用A...
4,2331 精英,2009/05/25,包括台灣的威邁思、威達超舜、全球一動等全球WiMAX業者將陸續在下半年商轉服務，將帶動WiM...
...,...,...,...
6028,3673 TPK-KY,2015/3/23,供應鏈傳出，受組裝與面板良率偏低影響，蘋果將Apple Watch 出貨目標砍半，將衝擊廣達...
6029,3673 TPK-KY,2015/3/11,蘋果10日共推出Apple Watch 與12吋Macbook 兩項產品，外資點出3 項意涵...
6030,3673 TPK-KY,2015/3/9,供應鏈表示，蘋果對於首款智慧穿戴裝置Apple Watch 的生產計畫相當謹慎，預計5 月後...
6031,3673 TPK-KY,2015/2/4,觸控面板大廠F-TPK宸鴻（3673）預計將於2/10舉行法人說明會。該公司歷經產 品組合及...


## 1-2 匯入正常公司新聞

### 1-2-1 匯入TFJ正常公司新聞

In [17]:
tej_normal = pd.read_csv('data/2018_2017_tej_normal_0423.csv')
tej_normal.head()

,公司碼,年月,則次,序號,新聞內容
0,9946 三發地產,2017/12/26,01,10,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,01,10,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...
2,6239 力成,2017/12/8,01,10,NAND Flash供需緊張紓解，對於手握非韓系各大記憶體廠封測訂單的力成來說影響有限。另外...
3,3008 大立光,2017/12/8,01,10,大立光狀告先進光妨害營業秘密，智慧財產法院針對民事訴訟一審判決先進光須賠償大立光15.22 ...
4,3008 大立光,2017/12/7,01,10,大立光(3008)控告先進光(3362)侵權案，一審判決先進光需賠償大立光15億元，對此，先...


In [18]:
len(tej_normal)

7269

### 1-2-2 匯入UDN正常公司新聞（0422：正常公司的UDN資料蒐集不完整，暫不採納）

In [19]:
# Folder_Path = '/home/michael/Documents/M10609101_Thesis/News/udn/normal/'
# file_list = os.listdir(Folder_Path)
# # file_list

In [20]:
# udn_normal = pd.DataFrame(columns=['company', 'title', 'time', 'content'])

# for file in file_list:
#     df_temp = pd.read_csv('News/udn/normal/' + file)
#     udn_normal = pd.concat([udn_normal, df_temp])
# udn_normal = udn_normal.drop('title', axis=1).dropna()
# udn_normal.head()

In [21]:
# len(udn_normal)

### 1-2-3 （針對UDN）擷取日期，把日期格式轉換為yyyy/mm/dd（0422：正常公司的UDN資料蒐集不完整，暫不採納）

In [22]:
# dates = []
# for time in udn_normal['time']:
#     s = time
#     match = re.search('\d+-\d+-\d+', s)
#     date = str(match.group(0))
#     date = date.replace('-', '/')
#     dates.append(date)
# udn_normal['date'] = dates
# # udn_normal.drop('time', axis=1, inplace=True)
# udn_normal.tail()

### 1-2-4 合併危機公司資料集為 normal_news

In [23]:
normal_news = pd.DataFrame(columns=['company', 'date', 'content'])
normal_news['company'] = tej_normal['公司碼']
normal_news['date'] = tej_normal['年月']
normal_news['content'] = tej_normal['新聞內容']
normal_news.head()

,company,date,content
0,9946 三發地產,2017/12/26,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...
2,6239 力成,2017/12/8,NAND Flash供需緊張紓解，對於手握非韓系各大記憶體廠封測訂單的力成來說影響有限。另外...
3,3008 大立光,2017/12/8,大立光狀告先進光妨害營業秘密，智慧財產法院針對民事訴訟一審判決先進光須賠償大立光15.22 ...
4,3008 大立光,2017/12/7,大立光(3008)控告先進光(3362)侵權案，一審判決先進光需賠償大立光15億元，對此，先...


In [24]:
# normal_news = pd.DataFrame(columns=['company', 'date', 'content'])
# company = udn_normal['company']
# company = company.append(tej_normal['公司碼'])
# date = udn_normal['date']
# date = date.append(tej_normal['年月'])
# content = udn_normal['content']
# content = content.append(tej_normal['新聞內容'])
# normal_news['company'] = company
# normal_news['date'] = date
# normal_news['content'] = content

In [25]:
# normal_news.head()

In [26]:
len(normal_news)

7269

In [27]:
normal_news.to_csv("data/tej_normal_news.csv", index=False)
normal_news = pd.read_csv("data/tej_normal_news.csv", encoding="utf-8")
normal_news

,company,date,content
0,9946 三發地產,2017/12/26,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...
2,6239 力成,2017/12/8,NAND Flash供需緊張紓解，對於手握非韓系各大記憶體廠封測訂單的力成來說影響有限。另外...
3,3008 大立光,2017/12/8,大立光狀告先進光妨害營業秘密，智慧財產法院針對民事訴訟一審判決先進光須賠償大立光15.22 ...
4,3008 大立光,2017/12/7,大立光(3008)控告先進光(3362)侵權案，一審判決先進光需賠償大立光15億元，對此，先...
...,...,...,...
7264,3494 誠研,2014/5/28,華碩(2357)、全家(5903)與誠研(3494)三方異業結盟，於今(28)日正式攜手推出...
7265,3494 誠研,2013/12/10,誠研（3494）昨（9）日發表自有品牌「Pringo」行動照片沖印機，宣告即日起 將透過神腦...
7266,3494 誠研,2013/12/10,誠研（3494）在杜拜經銷商進行大量庫存回補，加上行動相片沖印機微量出貨下，使得11月營收較...
7267,3494 誠研,2013/9/5,相片沖印機廠商誠研今（5）日公佈8月營收為2.00億元，月增41.84%、年增2.99%。該...


In [37]:
all_news = pd.concat([normal_news, distress_news])
all_news

,company,date,content
0,9946 三發地產,2017/12/26,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...
2,6239 力成,2017/12/8,NAND Flash供需緊張紓解，對於手握非韓系各大記憶體廠封測訂單的力成來說影響有限。另外...
3,3008 大立光,2017/12/8,大立光狀告先進光妨害營業秘密，智慧財產法院針對民事訴訟一審判決先進光須賠償大立光15.22 ...
4,3008 大立光,2017/12/7,大立光(3008)控告先進光(3362)侵權案，一審判決先進光需賠償大立光15億元，對此，先...
...,...,...,...
6028,3673 TPK-KY,2015/3/23,供應鏈傳出，受組裝與面板良率偏低影響，蘋果將Apple Watch 出貨目標砍半，將衝擊廣達...
6029,3673 TPK-KY,2015/3/11,蘋果10日共推出Apple Watch 與12吋Macbook 兩項產品，外資點出3 項意涵...
6030,3673 TPK-KY,2015/3/9,供應鏈表示，蘋果對於首款智慧穿戴裝置Apple Watch 的生產計畫相當謹慎，預計5 月後...
6031,3673 TPK-KY,2015/2/4,觸控面板大廠F-TPK宸鴻（3673）預計將於2/10舉行法人說明會。該公司歷經產 品組合及...


In [38]:
all_news['date'] = pd.to_datetime(all_news['date'])
all_news = all_news.sort_values(by=['date']).reset_index(drop=True) # 用post_time排序 # 在重設index
all_news

,company,date,content
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
2,3037 欣興,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
3,5469 瀚宇博,2007-01-01,台灣表面黏著科技（6278）受惠台灣近年面板蓬勃發展，去年成為最大的印刷電路板表面黏著科技廠...
4,2316 楠梓電,2007-01-01,台灣表面黏著科技（6278）受惠台灣近年面板蓬勃發展，去年成為最大的印刷電路板表面黏著科技廠...
...,...,...,...
13297,1611 中電,2017-12-28,公開資訊觀測站重大訊息公告 ...
13298,1611 中電,2017-12-29,本土照明大廠 （ ）處分台北市忠孝東路、成功段不動產，預計獲利9.32億元，挹注每股純益2....
13299,2474 可成,2017-12-29,可成旗下子公司可成科技( 宿遷) 及可功科技( 宿遷)28 日分別斥資6.2 億元及6.3 ...
13300,1216 統一,2017-12-29,（ ）旗下台灣 星巴克，確定明年1月起由 企業集團獨資經營，這代表 與美國星巴克的股權交易已...


In [39]:
all_news.to_csv("data/tej_udn_all_news.csv", index=False)
all_news = pd.read_csv("data/tej_udn_all_news.csv", encoding="utf-8")
all_news

,company,date,content
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
2,3037 欣興,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
3,5469 瀚宇博,2007-01-01,台灣表面黏著科技（6278）受惠台灣近年面板蓬勃發展，去年成為最大的印刷電路板表面黏著科技廠...
4,2316 楠梓電,2007-01-01,台灣表面黏著科技（6278）受惠台灣近年面板蓬勃發展，去年成為最大的印刷電路板表面黏著科技廠...
...,...,...,...
13297,1611 中電,2017-12-28,公開資訊觀測站重大訊息公告 ...
13298,1611 中電,2017-12-29,本土照明大廠 （ ）處分台北市忠孝東路、成功段不動產，預計獲利9.32億元，挹注每股純益2....
13299,2474 可成,2017-12-29,可成旗下子公司可成科技( 宿遷) 及可功科技( 宿遷)28 日分別斥資6.2 億元及6.3 ...
13300,1216 統一,2017-12-29,（ ）旗下台灣 星巴克，確定明年1月起由 企業集團獨資經營，這代表 與美國星巴克的股權交易已...


In [40]:
all_news.describe()

,company,date,content
count,13302,13302,13302
unique,240,3038,12746
top,2330 台積電,2008-01-17,中華信用評等公司昨日確認台塑集團四家核心公司，包括台塑、南亞塑膠、台化、台塑石化，長期企業信...
freq,613,20,4


In [41]:
all_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13302 entries, 0 to 13301
Data columns (total 3 columns):
company    13302 non-null object
date       13302 non-null object
content    13302 non-null object
dtypes: object(3)
memory usage: 311.9+ KB


In [42]:
all_news["len"] = all_news["content"].apply(len)
all_news

,company,date,content,len
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,1363
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,1358
2,3037 欣興,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,1364
3,5469 瀚宇博,2007-01-01,台灣表面黏著科技（6278）受惠台灣近年面板蓬勃發展，去年成為最大的印刷電路板表面黏著科技廠...,2080
4,2316 楠梓電,2007-01-01,台灣表面黏著科技（6278）受惠台灣近年面板蓬勃發展，去年成為最大的印刷電路板表面黏著科技廠...,2080
...,...,...,...,...
13297,1611 中電,2017-12-28,公開資訊觀測站重大訊息公告 ...,2557
13298,1611 中電,2017-12-29,本土照明大廠 （ ）處分台北市忠孝東路、成功段不動產，預計獲利9.32億元，挹注每股純益2....,418
13299,2474 可成,2017-12-29,可成旗下子公司可成科技( 宿遷) 及可功科技( 宿遷)28 日分別斥資6.2 億元及6.3 ...,113
13300,1216 統一,2017-12-29,（ ）旗下台灣 星巴克，確定明年1月起由 企業集團獨資經營，這代表 與美國星巴克的股權交易已...,697


In [43]:
all_news.describe()

,len
count,13302.000000
mean,565.182153
std,365.609065
min,2.000000
25%,281.000000
50%,497.000000
75%,761.000000
max,4032.000000


## 2-3 每間公司風險分布
使用意藍2015年風險資料集、經過BERT主題模型產生的model 將每篇文章輸出1*5的output

13302篇文章就有13302*5

In [47]:
company_list = all_news["company"].unique()
print(len(company_list))
company_list

240


array(['2316 楠梓電', '5469 瀚宇博', '3037 欣興', '1612 宏泰', '2498 宏達電',
       '1477 聚陽', '6168 宏齊', '2484 希華', '2534 宏盛', '6277 宏正', '2606 裕民',
       '2527 宏璟', '5305 敦南', '2376 技嘉', '3018 同開', '1236 宏亞', '2536 宏普',
       '3052 夆典', '1558 伸興', '1452 宏益', '2358 廷鑫', '3605 宏致', '2408 南亞科',
       '6271 同欣電', '3015 全漢', '4532 瑞智', '6152 百一', '2331 精英', '1532 勤美',
       '2325 矽品', '3682 亞太電', '8271 宇瞻', '2344 華邦電', '9934 成霖', '8150 南茂',
       '6196 帆宣', '1583 程泰', '3528 安馳', '2511 太子', '8021 尖點', '9911 櫻花',
       '3591 艾笛森', '4919 新唐', '4119 旭富', '1442 名軒', '1618 合機', '1414 東和',
       '1805 寶徠', '1451 年興', '2528 皇普', '2429 銘旺科', '1525 江申', '4414 如興',
       '9943 好樂迪', '2301 光寶科', '2472 立隆電', '3406 玉晶光', '1907 永豐餘',
       '3026 禾伸堂', '6285 啟碁', '2485 兆赫', '2461 光群雷', '2419 仲琦',
       '6281 全國電', '2439 美律', '2340 光磊', '3321 同泰', '2509 全坤建', '1323 永裕',
       '3058 立德', '3004 豐達科', '2330 台積電', '1904 正隆', '5608 四維航',
       '4906 正文', '2392 正崴', '2105 正新', '9921 巨大', '2506 太設', '1507 永大',
 

In [49]:
df_company = all_news[all_news['company'].str.contains("2316 楠梓電")]
df_company

,company,date,content,len
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,1363
4,2316 楠梓電,2007-01-01,台灣表面黏著科技（6278）受惠台灣近年面板蓬勃發展，去年成為最大的印刷電路板表面黏著科技廠...,2080
11,2316 楠梓電,2007-01-03,華通電腦（2313）、 子（ ）、燿華電子（2367）、欣興電子（3037）等四大手機印...,785
13,2316 楠梓電,2007-01-03,印刷電路板今年第一季面臨產業淡季，加上工作天數減少情況，普遍預估將較上一季減少10-15%，...,891
18,2316 楠梓電,2007-01-08,雖然HDI 製程目前多運用在供應給國際手機大廠的手機板中，但以多樣少量起家的柏承另闢戰場，以...,250
...,...,...,...,...
2992,2316 楠梓電,2008-11-18,上市櫃印刷電路板（PCB）廠商今年原本欲積極買進庫藏股護盤，但在近期股價依舊重挫後紛紛縮手，...,742
2993,2316 楠梓電,2008-11-18,（ ）：受惠下半年傳統旺季，帶動第三季轉虧為盈，公司除買進庫藏股護盤，董監事也小動作買進，...,171
3024,2316 楠梓電,2008-11-24,法人圈傳出南部PCB廠楠梓電(2316)裁員550人，為此該公司發言人呂淑芳回應表示，並沒有...,711
3038,2316 楠梓電,2008-11-26,市場傳出日月光無預警裁員千人，也有一種說法是高雄廠自10月起裁員10% 。惟日月光已對上述傳...,406


In [ ]:
df_year = 

In [ ]:
for company_name in company_list:
    